<a href="https://colab.research.google.com/github/PrateekR16/CSCI-544_Applied_NLP/blob/main/csci544_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install contractions

In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
class Dataset:
    RANDOM_STATE = 57
    DATA_PATH = "amazon_reviews_us_Office_Products_v1_00.tsv.gz"
    TEST_SPLIT = 0.2
    N_SAMPLES_EACH_CLASS = 100000
    NUM_TFIDF_FEATURES = 5000
    NUM_BOW_FEATURES = 8000

In [4]:
# %%bash
# cd "/content/drive/MyDrive/CSCI_544_NLP/HW1"
# curl -o amazon_reviews_us_Office_Products_v1_00.tsv.gz \
# https://web.archive.org/web/20201127142707if_/https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz

In [5]:
os.chdir("/content/drive/MyDrive/CSCI_544_NLP/HW1")

In [6]:
df_small = pd.read_csv(
    Dataset.DATA_PATH,
    sep='\t',
    on_bad_lines="skip",
    memory_map=True,
    nrows=10
)

In [7]:
df_small.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

In [8]:
df = pd.read_csv(
    Dataset.DATA_PATH,
    sep='\t',
    usecols=["review_body", "star_rating"],
    on_bad_lines="skip",
    memory_map=True,
)

<ipython-input-8-42625658a1fd>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [9]:
df.head()

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use


In [10]:
df.describe()

,star_rating,review_body
count,2640338,2640178
unique,13,2421350
top,5,good
freq,1459093,5176


In [11]:
df['star_rating'].unique()

array([5, 1, 4, 2, 3, '5', '1', '3', '4', '2', '2015-06-05', '2015-02-11',
       nan, '2014-02-14'], dtype=object)

In [12]:
df["star_rating"].value_counts()

,count
star_rating,
5,1459093
4,389627
1,286098
3,179870
2,129042
5,123784
4,28754
1,20894
3,13824


# Preprocess 'star_ratings'


In [13]:
df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce').astype('Int64')

df.dropna(inplace=True)

In [14]:
df['star_rating'].unique()

<IntegerArray>
[5, 1, 4, 2, 3]
Length: 5, dtype: Int64

In [15]:
df['star_rating'].value_counts()

,count
star_rating,
5,1582769
4,418358
1,306980
3,193683
2,138388


In [16]:
df['sentiment'] = df['star_rating'].apply(lambda x: 0 if x in [1, 2] else (1 if x in [4, 5] else None)).astype('Int64')

In [17]:
df.head()

,star_rating,review_body,sentiment
0,5,Great product.,1
1,5,What's to say about this commodity item except...,1
2,5,"Haven't used yet, but I am sure I will like it.",1
3,1,Although this was labeled as &#34;new&#34; the...,0
4,4,Gorgeous colors and easy to use,1


In [18]:
df.describe()

,star_rating,sentiment
count,2640178.0,2446495.0
mean,4.072484,0.817957
std,1.386981,0.38588
min,1.0,0.0
25%,4.0,1.0
50%,5.0,1.0
75%,5.0,1.0
max,5.0,1.0


In [19]:
df['sentiment'].unique()

<IntegerArray>
[1, 0, <NA>]
Length: 3, dtype: Int64

In [20]:
df['sentiment'].value_counts(dropna=False)

,count
sentiment,
1,2001127
0,445368
<NA>,193683


In [21]:
df = df.dropna(subset=['sentiment']).reset_index(drop=True)

In [22]:
df.shape

(2446495, 3)

In [23]:
desired_samples = Dataset.N_SAMPLES_EACH_CLASS
class_0 = df[df['sentiment'] == 0]
class_1 = df[df['sentiment'] == 1]
class_0_balanced = class_0.sample(n=desired_samples, random_state=44)
class_1_balanced = class_1.sample(n=desired_samples, random_state=44)

balanced_df = pd.concat([class_0_balanced, class_1_balanced]).sample(frac=1, random_state=42).reset_index(drop=True)

In [24]:
balanced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   star_rating  200000 non-null  Int64 
 1   review_body  200000 non-null  object
 2   sentiment    200000 non-null  Int64 
dtypes: Int64(2), object(1)
memory usage: 5.0+ MB


In [25]:
balanced_df['sentiment'].value_counts()

,count
sentiment,
1,100000
0,100000


In [26]:
balanced_df.head()

,star_rating,review_body,sentiment
0,5,delivered as promised,1
1,1,"The mount seem ok , but the brackets doesn't e...",0
2,5,.............considering I didn't really know ...,1
3,2,"The lanyard itself is of great quality, althou...",0
4,1,"Horrible pencil sharpener, doesn't work, all i...",0


# Step 2: Data Cleaning

In [27]:
import re
import unicodedata
import contractions

In [28]:
def unicode_to_ascii(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text)
                   if unicodedata.category(c) != 'Mn')

def expand_contractions(text):
    return contractions.fix(text)

def clean_review(text):
    text = unicode_to_ascii(text.lower().strip())
    text = re.sub(r"[a-zA-Z0-9_\-\.]+@[a-zA-Z0-9_\-\.]+\.[a-zA-Z]{2,5}", " ", text)
    text = re.sub(r"\bhttps?:\/\/\S+|www\.\S+", " ", text)
    text = re.sub(r"<.*?>", "", text)
    text = expand_contractions(text)
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    text = re.sub(r"[^a-zA-Z\s]+", "", text)
    text = re.sub(" +", " ", text)
    return text.strip()

clean_review_vectorized = np.vectorize(clean_review)

In [29]:
mean_before_cleaning = balanced_df['review_body'].apply(len).mean()

balanced_df['review_body'] = balanced_df['review_body'].apply(clean_review_vectorized)
balanced_df = balanced_df.loc[balanced_df["review_body"].str.strip() != ""]
mean_after_cleaning = balanced_df['review_body'].apply(len).mean()

print(f"Mean length of reviews before cleaning: {mean_before_cleaning}")
print(f"Mean length of reviews after cleaning: {mean_after_cleaning}")

Mean length of reviews before cleaning: 316.493865
Mean length of reviews after cleaning: 301.3579452808483


In [30]:
balanced_df['sentiment'].value_counts()

,count
sentiment,
0,99985
1,99945


In [31]:
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [32]:
original_stopwords = set(stopwords.words('english'))
neg_words = ['no', 'not', 'nor', 'neither', 'none', 'never', 'nobody', 'nowhere']
custom_stopwords = [word for word in original_stopwords if word not in neg_words]

#For custom stopwords
pattern = re.compile(r'\b('+r'|'.join(custom_stopwords)+r')\b\s*')

In [33]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word,pos='v') for word in words]
    return ' '.join(lemmatized_words)

def preprocess_text(text):
    text = pattern.sub('',text)
    text = lemmatize_text(text)
    return text

preprocess_text_vect = np.vectorize(preprocess_text)

In [34]:
balanced_df.head()

,star_rating,review_body,sentiment
0,5,delivered as promised,1
1,1,the mount seem ok but the brackets does not ex...,0
2,5,considering i did not really know what the hec...,1
3,2,the lanyard itself is of great quality althoug...,0
4,1,horrible pencil sharpener does not work all it...,0


In [35]:

avg_len_before_preprocess = mean_after_cleaning
balanced_df["review_body"] = balanced_df["review_body"].apply(preprocess_text_vect)

avg_len_after_preprocess = balanced_df["review_body"].apply(len).mean()

print(f'Avg. Length of Reviews Before Preprocessing: {avg_len_before_preprocess:.2f} characters')
print(f'Avg. Length of Reviews After Preprocessing: {avg_len_after_preprocess:.2f} characters')


Avg. Length of Reviews Before Preprocessing: 301.36 characters
Avg. Length of Reviews After Preprocessing: 184.82 characters


In [36]:
balanced_df['sentiment'].value_counts()

,count
sentiment,
0,99985
1,99945


In [37]:
balanced_df.head()

,star_rating,review_body,sentiment
0,5,deliver promise,1
1,1,mount seem ok bracket not extend long enough r...,0
2,5,consider not really know heck figure however w...,1
3,2,lanyard great quality although not release opt...,0
4,1,horrible pencil sharpener not work break tip g...,0


In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

vectorizer = TfidfVectorizer()
tfidf= vectorizer.fit_transform(balanced_df['review_body'])
labels = balanced_df['sentiment']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(tfidf, labels, test_size=0.2, random_state=42)

In [40]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(159944, 81876)
(39986, 81876)
(159944,)
(39986,)
(199930,)


In [41]:
print(tfidf.shape)

(199930, 81876)


In [42]:
balanced_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199930 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   star_rating  199930 non-null  Int64 
 1   review_body  199930 non-null  object
 2   sentiment    199930 non-null  Int64 
dtypes: Int64(2), object(1)
memory usage: 6.5+ MB


In [43]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
model = Perceptron(max_iter=10000, tol=1e-3, random_state=42)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

print("Training Metrics:")
print(f"Accuracy   : {train_accuracy:.4f}")
print(f"Precision  : {train_precision:.4f}")
print(f"Recall     : {train_recall:.4f}")
print(f"F1-Score   : {train_f1:.4f}")

print("\nTesting Metrics:")
print(f"Accuracy   : {test_accuracy:.4f}")
print(f"Precision  : {test_precision:.4f}")
print(f"Recall     : {test_recall:.4f}")
print(f"F1-Score   : {test_f1:.4f}")

Training Metrics:
Accuracy   : 0.9208
Precision  : 0.9179
Recall     : 0.9241
F1-Score   : 0.9210

Testing Metrics:
Accuracy   : 0.8700
Precision  : 0.8697
Recall     : 0.8710
F1-Score   : 0.8703


In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

print("Training Metrics:")
print(f"Accuracy   : {train_accuracy:.4f}")
print(f"Precision  : {train_precision:.4f}")
print(f"Recall     : {train_recall:.4f}")
print(f"F1-Score   : {train_f1:.4f}")

print("\nTesting Metrics:")
print(f"Accuracy   : {test_accuracy:.4f}")
print(f"Precision  : {test_precision:.4f}")
print(f"Recall     : {test_recall:.4f}")
print(f"F1-Score   : {test_f1:.4f}")

Training Metrics:
Accuracy   : 0.9184
Precision  : 0.9217
Recall     : 0.9144
F1-Score   : 0.9180

Testing Metrics:
Accuracy   : 0.9079
Precision  : 0.9109
Recall     : 0.9046
F1-Score   : 0.9078


In [45]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = MultinomialNB()

model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

print("Training Metrics:")
print(f"Accuracy   : {train_accuracy:.4f}")
print(f"Precision  : {train_precision:.4f}")
print(f"Recall     : {train_recall:.4f}")
print(f"F1-Score   : {train_f1:.4f}")

print("\nTesting Metrics:")
print(f"Accuracy   : {test_accuracy:.4f}")
print(f"Precision  : {test_precision:.4f}")
print(f"Recall     : {test_recall:.4f}")
print(f"F1-Score   : {test_f1:.4f}")

Training Metrics:
Accuracy   : 0.8782
Precision  : 0.9002
Recall     : 0.8504
F1-Score   : 0.8746

Testing Metrics:
Accuracy   : 0.8630
Precision  : 0.8840
Recall     : 0.8361
F1-Score   : 0.8594


In [47]:
from sklearn.svm import SVC,LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = LinearSVC(max_iter=1000, tol=1e-3, random_state=42)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)

print("Training Metrics:")
print(f"Accuracy   : {train_accuracy:.4f}")
print(f"Precision  : {train_precision:.4f}")
print(f"Recall     : {train_recall:.4f}")
print(f"F1-Score   : {train_f1:.4f}")

print("\nTesting Metrics:")
print(f"Accuracy   : {test_accuracy:.4f}")
print(f"Precision  : {test_precision:.4f}")
print(f"Recall     : {test_recall:.4f}")
print(f"F1-Score   : {test_f1:.4f}")

Training Metrics:
Accuracy   : 0.9401
Precision  : 0.9431
Recall     : 0.9368
F1-Score   : 0.9399

Testing Metrics:
Accuracy   : 0.9043
Precision  : 0.9050
Recall     : 0.9039
F1-Score   : 0.9044
